In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
url = "https://www.optioncarriere.tn/emploi?s=&l="

In [4]:
def get_jobs_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    job_articles = soup.find_all("article", class_="job clicky")
    links = []
    
    for job in job_articles:
        if "data-url" in job.attrs:
            links.append("https://www.optioncarriere.tn" + job["data-url"])
    return links

In [14]:
def get_job_details(link):
    response = requests.get(link)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")
    
    job_details = {}
    job_details["Title"] = soup.find("h1").text.strip() if soup.find("h1") else "N/A"
    job_details["Company"] = soup.find("p", class_="company").text.strip() if soup.find("p", class_="company") else "N/A"
    job_details["Location"] = soup.find("span").text.strip() if soup.find("span") else "N/A"
    job_details["Description"] = soup.find("section", class_="content").text.strip() if soup.find("section", class_="content") else "N/A"
    
    date_badge = soup.find("span", class_="badge badge-r badge-s")
    job_details["Date Posted"] = date_badge.text.strip() if date_badge else "N/A"    
    
    job_details["Link"] = link
    
    return job_details

In [ ]:
def scrape_optioncarriere(base_url):
    all_jobs = []
    current_page = 1
    
    while current_page <= 2:
        job_links = get_jobs_links(base_url)
        
        if not job_links:
            print("No more job offers found. Scraping complete.")
            break
        
        for link in job_links:
            try:
                job_details = get_job_details(link)
                all_jobs.append(job_details)
            except Exception as e:
                print(f"Failed to scrape job details from {link}: {e}")
        
        soup = BeautifulSoup(requests.get(base_url).text, "html.parser")
        
        next_button = soup.find("button", {"class": "ves-control ves-add btn btn-r btn-primary-inverted next"})
        
        if next_button and "data-value" in next_button.attrs:
            next_page_value = next_button["data-value"]
            base_url = f"https://www.optioncarriere.tn/emploi?s=&l=&p={next_page_value}"
            current_page += 1  
        else:
            print("No more pages found. Ending scraping.")
            break
    
    return all_jobs


In [20]:
jobs = scrape_optioncarriere(url)

for job in jobs:
    formatted_job = {
        "Title": job["Title"],
        "Company": job["Company"],
        "Location": job["Location"],
        "Date Posted": job["Date Posted"],
        "Description": job["Description"],
        "Link": job["Link"],
    }
    print(formatted_job)

Scraping page 1...
Scraping page 2...
{'Title': 'Préparateur Préparatrice en Pharmacie à Hammamet', 'Company': 'Pharmacie Yasmine Bel Hadj Hassine, Hammamet (en face de la tour bleue)', 'Location': "Recherche d'emploi", 'Date Posted': 'Il y a 1 jour', 'Description': "Les missions principales seront : - Vérifier les livraisons - Réceptionner les commandes, ranger et classer les produits reçus - Délivrer les ordonnances, veiller à leur compréhension et à l'absence d'intéraction médicamenteuse, et apporter les conseils associés - Avoir un rôle d'écoute et de conseil afin de favoriser l'amélioration ou le maintien de l'état de santé des personnes - Conseiller des produits de parapharmacie  Qualités: - Être vigilant,\xa0rigoureux,\xa0méthodique,\xa0organisé\xa0et posséder une capacité de concentration à toute épreuve. - Avoir un esprit scientifique et une excellente connaissance des médicaments - Avoir le sens du contact humain, être\xa0discret et patient\xa0pour écouter, conseiller et rass